In [1]:
import pandas as pd
import requests

In [2]:
#Import CSVs and Creating DataFrames
housing = pd.read_csv('reduced_realty_df.csv')
housing

careers_url = "web_scrape_top-100-careers.html"
careers_mega = pd.read_html(careers_url)
careers = careers_mega[2]
careers

employment_data = pd.read_excel("state_M2021_dl.xlsx")
employment_data

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,17.91,28.68,39.79,18630,24720,37250,59660,82760,NaN,NaN
1,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,45.51,61.25,87.58,46900,62790,94650,127400,182160,NaN,NaN
2,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1011,Chief Executives,...,61.14,82.09,#,59190,94360,127170,170750,#,NaN,NaN
3,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,...,46.28,68.75,#,46220,62260,96270,143000,#,NaN,NaN
4,1,Alabama,2,AL,0,Cross-industry,cross-industry,1235,11-1031,Legislators,...,*,*,*,16930,17310,17840,31300,56420,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37575,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7051,Industrial Truck and Tractor Operators,...,13.86,17.17,18.41,22830,26700,28820,35720,38290,NaN,NaN
37576,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7061,Cleaners of Vehicles and Equipment,...,13.89,14.08,17.23,22260,24010,28890,29280,35840,NaN,NaN
37577,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7062,"Laborers and Freight, Stock, and Material Move...",...,14.14,16.19,18.5,23590,24560,29410,33680,38480,NaN,NaN
37578,78,Virgin Islands,3,VI,0,Cross-industry,cross-industry,1235,53-7064,"Packers and Packagers, Hand",...,11.92,12.16,14.39,23460,24060,24790,25300,29940,NaN,NaN


In [3]:
housing.columns

Index(['Unnamed: 0', 'status', 'price', 'city', 'state', 'sold_date',
       'house_size'],
      dtype='object')

In [5]:
#Groupby and Rename Columns
housing_by_state = housing.groupby('state').mean()
housing_by_state = housing_by_state[['price', 'house_size']]
housing_by_state.columns = ['Average Price', 'Average Square Feet']
housing_by_state

employment_df = employment_data[['OCC_TITLE', 'A_MEAN', 'AREA_TITLE']]
employment_df

,Average Price,Average Square Feet
state,,
Connecticut,4.275005e+05,2031.921645
Massachusetts,1.110066e+06,2283.397881
New Hampshire,5.964610e+05,2269.087580
New Jersey,3.334900e+05,1500.000000
New York,7.905664e+05,2858.399378
Puerto Rico,4.336704e+05,1941.929972
Rhode Island,4.227907e+05,2269.601562
Vermont,4.759933e+05,2527.551990
Virgin Islands,1.453273e+06,3963.343936


In [6]:
# Edit Occupation Names
employment_df = employment_df.replace(to_replace='Dentists, General', value='Dentist')
employment_df = employment_df.replace(to_replace='Registered Nurses', value='Registered Nurse')
employment_df = employment_df.replace(to_replace='Pharmacists', value='Pharmacist')
employment_df = employment_df.replace(to_replace='Computer Systems Analysts', value='Computer Systems Analyst')
employment_df = employment_df.replace(to_replace='Family Medicine Physicians', value='Physician')

In [7]:
#TODO Merge Tables
    # Table 1 Jobs: Job Title, Location, Salary (Range), (Unemployment)
    # Table 2 Housing: Housing Price, Location, Rooms, Acreage, Sq Ft-age
Jobs_df = careers.merge(employment_df, left_on=1, right_on='OCC_TITLE', how='inner')
Jobs_df = Jobs_df[[1, 4, 'A_MEAN', 'AREA_TITLE']]
Jobs_df.columns = ['Occupation', 'National Unemployment Rate', 'State Mean Income', 'State']
Jobs_df

,Occupation,National Unemployment Rate,State Mean Income,State
0,Dentist,0.7%,176910,Alabama
1,Dentist,0.7%,168390,Alaska
2,Dentist,0.7%,175030,Arizona
3,Dentist,0.7%,184540,Arkansas
4,Dentist,0.7%,165950,California
...,...,...,...,...
261,Physician,0.7%,227500,West Virginia
262,Physician,0.7%,261250,Wisconsin
263,Physician,0.7%,282550,Wyoming
264,Physician,0.7%,182210,Guam


In [ ]:
#TODO Export into PGAdmin4